# Olivia Radcliffe
You are required to design a simple 1-D CNN implementation for a DNA sequence classification task using Python and TensorFlow/Keras. For this problem, the model is trained to predict whether a given sequence contains the motif patern 'ATCGATCG'You can modify the motif_patern variable and the length of the random_sequence to test the model with diﬀerent motifs and sequence lengths. The accuracy on the test set is printed after training the model. Adjust the hyperparameters, such as the number of filters, kernel size, and training epochs, based on your specific requirements and dataset characteristics.

# Imports

In [4]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping 


# Function to generate a random DNA sequence

In [5]:
# Generate a random DNA sequence
np.random.seed(42)

def generate_random_sequence(sequence_length):
    nucleotides = ['A', 'C', 'G', 'T']
    return ''.join(np.random.choice(nucleotides, sequence_length))

# Create Dataset

In [6]:
def createDataset(motif_pattern, sequence_length, num_samples): 

    positive_sequences = []
    negative_sequences = []

    # -- Create negative samples without the motif --
    while len(negative_sequences) < num_samples:
        # Generate a random sequence
        random_sequence = generate_random_sequence(sequence_length)

        # Check if the sequence contains the motif
        if motif_pattern not in random_sequence:
            negative_sequences.append(random_sequence)
        else:
            positive_sequences.append(random_sequence)

    # -- Create positive samples with the motif --
    while len(positive_sequences) < num_samples:
        # Define a random starting point
        start = np.random.randint(0, sequence_length - len(motif_pattern))

        # Generate a random sequence
        positive_seq = generate_random_sequence(sequence_length)

        # Check if the sequence contains the motif
        if motif_pattern not in random_sequence:
            # Insert the motif into the random sequence
            positive_seq = positive_seq[0:start] + motif_pattern + positive_seq[start + len(motif_pattern):]

        # Append the sequence
        positive_sequences.append(positive_seq)


    # -- Combine positive and negative samples --
    sequences = positive_sequences + negative_sequences
    # Create labels
    labels = ["positive"] * len(positive_sequences) + ["negative"] * len(negative_sequences)

    # -- Convert DNA sequences to one-hot encoding --
    def dna_to_one_hot(sequence):
        mapping = {'A': [1, 0, 0, 0], 'C': [0, 1, 0, 0], 'G': [0, 0, 1, 0], 'T': [0, 0, 0, 1]}
        return np.array([mapping[nuc] for nuc in sequence])

    one_hot_sequences = np.array([dna_to_one_hot(seq) for seq in sequences])

    # Encode labels
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(labels)
    encoded_labels = to_categorical(encoded_labels)

    # -- Split the data into training and testing sets --
    one_hot_X_train, one_hot_X_test, y_train, y_test = train_test_split(one_hot_sequences, encoded_labels, test_size=0.15, random_state=42)


    return one_hot_X_train, one_hot_X_test, y_train, y_test

# Build, compile and train the CNN model

In [7]:
def build_and_train(one_hot_X_train, y_train, sequence_length):

    np.random.seed(4)

    # Build the model
    model = Sequential()
    model.add(Conv1D(64, 5, activation='relu', input_shape=(sequence_length, 4)))

    # Added another conv layer since with one I achieved 0.5 accuracy
    model.add(Conv1D(64, 5, activation='relu'))
    
    # Experimented with dropout - improved results when 2 conv layers
    model.add(Dropout(0.3))

    # GlobalAveragePooling1D worked much better than MaxPooling1D
    #model.add(MaxPooling1D(2))
    model.add(GlobalAveragePooling1D())

    model.add(Flatten())
    
    # Added a dense layer to improve accuracy by interpreting the features before classification
    model.add(Dense(64, activation='relu'))
    model.add(Dense(2, activation='softmax')) # 2 classes

    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Train the model
    early_stopping = EarlyStopping(monitor='val_loss', patience=4) # Reduced patience due to long training time
    model.fit(one_hot_X_train, y_train, batch_size=32, epochs=50, validation_split=0.3, callbacks=[early_stopping], verbose=0)

    return model



# Evaluate the model

In [8]:

def testModel(motif_pattern, sequence_length, num_samples):

    print("Parameters: "
        + "\n\tMotif Pattern: " + motif_pattern
        + "\n\tSequence Length: " + str(sequence_length)
        + "\n\tNumber of Samples: " + str(num_samples))

    # Create the dataset
    one_hot_X_train, one_hot_X_test, y_train, y_test = createDataset(motif_pattern, sequence_length, num_samples)
    # Build and train the model
    model = build_and_train(one_hot_X_train, y_train, sequence_length)
    # Evaluate the model
    loss, accuracy = model.evaluate(one_hot_X_test, y_test)
    print(f'Test Accuracy: {accuracy * 100:.2f}%')

# Testing Parameters

Took around 4 mins to run on my laptop

Results differ to to the ranodmization of the dataset.

In [10]:
testModel('ATCGATCG', 1000, 2000)

testModel('ATCGATCG', 100, 500)

testModel('AATCG', 100, 500)

testModel('AATCG', 50, 100)

testModel('AACG', 100, 500)

Parameters: 
	Motif Pattern: ATCGATCG
	Sequence Length: 1000
	Number of Samples: 2000


2023-12-04 07:43:15.222583: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-04 07:43:16.644743: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


19/19 [==============================] - 0s 11ms/step - loss: 0.0022 - accuracy: 1.0000
Test Accuracy: 100.00%
Parameters: 
	Motif Pattern: ATCGATCG
	Sequence Length: 100
	Number of Samples: 500


2023-12-04 07:44:26.225948: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-04 07:44:26.776665: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


5/5 [==============================] - 0s 15ms/step - loss: 0.0072 - accuracy: 1.0000
Test Accuracy: 100.00%
Parameters: 
	Motif Pattern: AATCG
	Sequence Length: 100
	Number of Samples: 500


...

5/5 [==============================] - 0s 15ms/step - loss: 0.0403 - accuracy: 0.9933
Test Accuracy: 99.33%
Parameters: 
	Motif Pattern: AATCG
	Sequence Length: 50
	Number of Samples: 100


2023-12-04 07:44:58.694942: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-04 07:44:59.095676: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 54ms/step - loss: 0.3637 - accuracy: 0.8667
Test Accuracy: 86.67%
Parameters: 
	Motif Pattern: AACG
	Sequence Length: 100
	Number of Samples: 500


2023-12-04 07:45:03.820452: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-12-04 07:45:04.398981: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


5/5 [==============================] - 0s 15ms/step - loss: 0.0281 - accuracy: 0.9933
Test Accuracy: 99.33%
